## 安裝及載入jieba與gensim下之word2vec模組

In [ ]:
!pip install jieba
!pip install gensim

步驟一：先將檔案中已訓練好的詞向量放入名為word_vecs這個詞典中

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np # 安裝numpy

dim = 0 #假設一個變數名字是維度
word_vecs= {}
# 開啟詞向量檔案
with open('cna.cbow.cwe_p.tar_g.512d.0.txt') as f:
  for line in f:
    # 假設我們的詞向量有512維度
    # 由word以及其向量數值共513行
    # 以空格分隔組成詞向量檔案中一行
    tokens = line.strip().split() #依空白作為分隔

    # 第一行是兩個整數，分別代表有幾個詞向量，以及詞向量維度
    if len(tokens) == 2:
      dim = int(tokens[1])
      continue
    
    word = tokens[0] 
    vec = np.array([ float(t) for t in tokens[1:] ])
    word_vecs[word] = vec

# 之後可以從word_vecs這個dict中取得詞向量

In [ ]:
word_vecs["企業"]

In [ ]:
import jieba
import numpy as np
dialogue = "台南市長是哪一個政黨"
answers = [
  "今天天氣是晴天",  
  "台南市長是民進黨",
  "我喜歡吃美食", 
  "我喜歡打羽球",
  "我就讀中信金融管理學院",
  "台南最有名的夜市式花園夜市"]

emb_cnt = 0 #計算一個句子有幾個詞，以作為計算平均值之用
avg_dlg_emb = np.zeros((dim,))
# jieba.cut 會把dialogue作分詞
# 對於有在word_vecs裡面的詞我們才把它取出
# 最後詞向量加總取平均，作為句子的向量表示
for word in jieba.cut(dialogue):
  if word in word_vecs:
    avg_dlg_emb += word_vecs[word]
    emb_cnt += 1
avg_dlg_emb /= emb_cnt

emb_cnt = 0
max_idx = -1
max_sim = -10
# 在六個回答中，每個答句都取詞向量平均作為向量表示
# 我們選出與dialogue句子向量表示cosine similarity最高的短句
for idx,ans in enumerate(answers):
  avg_ans_emb = np.zeros((dim,))
  for word in jieba.cut(ans):
    if word in word_vecs:
      avg_ans_emb += word_vecs[word]
      emb_cnt += 1
  sim = np.dot(avg_dlg_emb, avg_ans_emb) / np.linalg.norm(avg_dlg_emb) / np.linalg.norm(avg_ans_emb)
#算出相似性
  print("Ans#%d: %f" % (idx+1, sim))
  if sim > max_sim:
    max_idx = idx+1
    max_sim = sim

print("Answer:%d" % max_idx)
print(answers[max_idx-1])